In [1]:
import sys
import os

# Go one level up from generation/ to project root
PROJECT_ROOT = os.path.abspath("..")
sys.path.insert(0, PROJECT_ROOT)

print("Project root added to path:", PROJECT_ROOT)



Project root added to path: c:\Users\Sanya Kulthia\OneDrive\Desktop\sans_bot


In [ ]:
from embeddings.embed import final_query


c:\Users\Sanya Kulthia\OneDrive\Desktop\sans_bot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#test query
print(final_query("मूर्खभृत्यस्य कथा"))


[{'score': 0.64, 'language': 'sa', 'text': 'वरम् भृत्यविहिनस्य जिवितम् श्रमपूरितम् । मूर्खभृत्यस्य संसर्गात् सर्वम् कार्यम् विनश्यति ॥', 'section_id': 1}, {'score': 0.526, 'language': 'sa', 'text': 'मूर्खभृत्यस्य', 'section_id': 1}]


In [4]:
def build_context(retrieved_chunks):
    """
    Input: list of dicts from final_query()
    Output: clean context string
    """
    context_blocks = []

    for chunk in retrieved_chunks:
        context_blocks.append(chunk["text"])

    return "\n\n".join(context_blocks)


In [15]:
def build_prompt(query, context):
    return f"""
त्वं संस्कृत-आधारितः Retrieval-Augmented Generation (RAG) सहायः असि।

तव कार्यम्— अधोलिखितात् प्रदत्त-सन्दर्भात् (Context) एव
उपयोक्तृप्रश्नस्य यथार्थं सुसम्बद्धं च उत्तरं दातुम्।

नियमाः—
1. केवलं प्रदत्त-सन्दर्भस्य आधारेण एव उत्तरं देहि।
2. स्वकल्पितं, बाह्यं, असन्दर्भितं वा ज्ञानं न प्रयोजय।
3. यदि सन्दर्भे उत्तरं न लभ्यते, तर्हि स्पष्टं वद—
   "दत्तसन्दर्भे उत्तरं न उपलब्धम्।"
4. यदि प्रश्नः संस्कृते वा लिप्यन्तरितसंस्कृते अस्ति,
   तर्हि उत्तरं केवलं संस्कृते एव देहि।
5. यदि प्रश्नः स्पष्टतया आङ्ग्लभाषायां (English) अस्ति,
   तर्हि उत्तरं आङ्ग्लभाषायां एव देहि।
6. उत्तरं संक्षिप्तं, स्पष्टं, व्याकरणशुद्धं च भवेत्।

सन्दर्भः:
----------------
{context}
----------------

प्रश्नः:
{query}

उत्तरम्:
"""


In [16]:
import requests
import json

def generate_with_ollama(prompt, model="gemma3:4b"):
    url = "http://localhost:11434/api/generate"

    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()

    return response.json()["response"]


In [17]:
def rag_chat(query):
    # 1. Retrieve
    retrieved = final_query(query)

    if not retrieved:
        return "दत्तसन्दर्भे उत्तरं न उपलब्धम्।"

    # 2. Build context
    context = build_context(retrieved)

    # 3. Build prompt
    prompt = build_prompt(query, context)

    # 4. Generate answer
    answer = generate_with_ollama(prompt)

    return answer


In [ ]:
response = rag_chat("मूर्खभृत्यस्य संसर्गः सर्वं कार्यं विनश्यति” इत्यस्य अर्थं वद।")
print(response)


दत्तसन्दर्भे उत्तरं न उपलब्धम्।

